<a href="https://colab.research.google.com/github/Kunal2873/music_generator/blob/main/music_generation.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
! pip install music21
!pip install tensorflow

In [ ]:
!pip show webcolors

Name: webcolors
Version: 24.11.1
Summary: A library for working with the color formats defined by HTML and CSS.
Home-page: 
Author: James Bennett
Author-email: 
License: BSD-3-Clause
Location: /usr/local/lib/python3.11/dist-packages
Requires: 
Required-by: music21


In [ ]:
#importing the neccessary libraries
# data flair automatic music generation project
import numpy as np
import random
from music21 import*
import glob
from tqdm import tqdm
from tensorflow.keras.layers import LSTM,Dense,Input,Dropout
from tensorflow.keras.models import Model,Sequential,load_model
from sklearn.model_selection import train_test_split
from  tensorflow import keras

In [ ]:
import zipfile

with zipfile.ZipFile("/content/archive (5).zip", 'r') as zip_ref:
    zip_ref.extractall("Classical_Music_MIDI")


In [ ]:
def read_files(file):
 notes=[]
 notes_to_parse=None
 #parse the midi file
 midi=converter.parse(file)
 #seperate all instruments from the file
 instrmt=instrument.partitionByInstrument(midi)
 for part in instrmt.parts:
 #fetch data only of Piano instrument
  if 'Piano' in str(part):
   notes_to_parse=part.recurse()
 #iterate over all the parts of sub stream elements
 #check if element's type is Note or chord
 #if it is chord split them into notes
 for element in notes_to_parse:
  if type(element)==note.Note:
    notes.append(str(element.pitch))
  elif type(element)==chord.Chord:
    notes.append('.'.join(str(n) for n in element.normalOrder))
#return the list of notes
 return notes

file_path=['schubert']
all_files=glob.glob('Classical_Music_MIDI/'+file_path[0]+'/*.mid',recursive=True)

notes_array=np.array([read_files(i) for i in tqdm(all_files,position=0,leave=True)],dtype=object) #--> error in the code below is this correct version
# notes_list = [read_files(i) for i in tqdm(all_files, position=0, leave=True)]







  0%|          | 0/29 [00:00<?, ?it/s]/usr/local/lib/python3.11/dist-packages/music21/midi/translate.py:874: TranslateWarning: Unable to determine instrument from <music21.midi.MidiEvent SEQUENCE_TRACK_NAME, track=4, channel=None, data=b'Copyright \xa9 1997 by Bernd Krueger'>; getting generic Instrument
  warnings.warn(
  3%|▎         | 1/29 [00:01<00:31,  1.14s/it]/usr/local/lib/python3.11/dist-packages/music21/midi/translate.py:874: TranslateWarning: Unable to determine instrument from <music21.midi.MidiEvent SEQUENCE_TRACK_NAME, track=5, channel=None, data=b'Copyright \xa9 2009 by Bernd Krueger'>; getting generic Instrument
  warnings.warn(
 10%|█         | 3/29 [00:25<03:34,  8.26s/it]/usr/local/lib/python3.11/dist-packages/music21/midi/translate.py:874: TranslateWarning: Unable to determine instrument from <music21.midi.MidiEvent SEQUENCE_TRACK_NAME, track=4, channel=None, data=b'Copyright \xa9 2004 by Bernd Kr\xfcger'>; getting generic Instrument
  warnings.warn(
 21%|██        |

exploring the dataset

In [ ]:
notess= sum(notes_array,[])
unique_notes=list(set(notess))
print("unique notes:",len(unique_notes))

freq=dict(map(lambda x:(x,notess.count(x)),unique_notes))


for i in range(30,100,20):
  print(i,":",len(list(filter(lambda x:x[1]>=i,freq.items()))))

unique notes: 95
30 : 17
50 : 10
70 : 5
90 : 3


In [ ]:
freq_notes=dict(filter(lambda x:x [1]>=50,freq.items()))
new_notes=[[i for i in j if i in freq_notes] for j in notes_array]
#new_notes


In [ ]:
# dictionary having keys as note index and value as note
ind2note=dict(enumerate(freq_notes))
# dictionary having key as note and value as note index
note2ind=dict(map(reversed,ind2note.items()))

In [ ]:
# timestep
timestep=50
#store values of input and output
x=[];y=[]
for i in new_notes:
  for j in range(0,len(i)-timestep):
    # input will be current index +timestep
    #output will be the next index after timestep
    inp=i[j:j+timestep]; out=i[j+ timestep]
# append the index value of the respective notes
    x.append(list(map(lambda x:note2ind[x],inp)))
    y.append(note2ind[out])
x_new=np.array(x)
y_new=np.array(y)



training and testing the sets

In [ ]:
x_new=np.reshape(x_new,(len(x_new),timestep,1))
y_new=np.reshape(y_new,(-1,1))

x_train,x_test,y_train,y_test=train_test_split(x_new,y_new,test_size=0.2,random_state=42)

constructing the model

In [ ]:
model=Sequential()
#model of 2 stacked LSTM layers with the latent dimensoin of 256
model.add(LSTM(256,return_sequences=True,input_shape=(x_new.shape[1],x_new.shape[2])))


model.add(Dropout(0.2))
model.add(LSTM(256))
model.add(Dropout(0.2))
model.add(Dense(256,activation='relu'))
#fully connected layer for the output with softmax activation

model.add(Dense(len(note2ind),activation='softmax'))
model.summary()

/usr/local/lib/python3.11/dist-packages/keras/src/layers/rnn/rnn.py:200: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(**kwargs)


Model: "sequential"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ lstm (LSTM)                     │ (None, 50, 256)        │       264,192 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout (Dropout)               │ (None, 50, 256)        │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ lstm_1 (LSTM)                   │ (None, 256)            │       525,312 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout_1 (Dropout)             │ (None, 256)            │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense (Dense)                   │ (None, 256)            │        65,792 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_1 (Dense)                 │ (None, 10)             │         2,570 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 857,866 (3.27 MB)

 Trainable params: 857,866 (3.27 MB)

 Non-trainable params: 0 (0.00 B)

In [ ]:
# # save the model
model.save("s2s.keras")

In [ ]:
# model.save("music_model_tf", save_format="tf")


In [ ]:
# model.export("music_model_tf")  # ✅ Works in Keras 3
# model = keras.models.load_model("music_model_tf")


In [ ]:
model =load_model('s2s.keras')
index=np.random.randint(0,len(x_test)-1)
music_pattern=x_test[index]
out_pred=[]

for i in range(200):
  music_pattern=music_pattern.reshape(1,len(music_pattern),1)
  pred_index=np.argmax(model.predict(music_pattern))
  out_pred.append(ind2note[pred_index])
  music_pattern=np.append(music_pattern,pred_index)
  music_pattern=music_pattern[1:]

1/1 ━━━━━━━━━━━━━━━━━━━━ 1s 579ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 31ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 32ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 33ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 31ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 30ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 34ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 32ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 32ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 31ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 30ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 33ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 31ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 33ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 31ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 30ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 32ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 32ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 32ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 31ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 34ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 35ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 32ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 35ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 32ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 30ms/step
1/1 ━━━━━━━

In [ ]:
# saving the file
output_notes=[]
for offset,pattern in enumerate(out_pred):
  if('.'in pattern)or pattern.isdigit():
    notes_in_chord=pattern.split('.')
    notes=[]

    for current_note in notes_in_chord:
      i_curr_note=int(current_note)
      new_note=note.Note(i_curr_note)
      new_note.storedInstrument=instrument.Piano()
      notes.append(new_note)
      new_chord=chord.Chord(notes)
      new_chord.offset=offset
      out_notes.append(new_chord)


  else:
    new_note=note.Note(pattern)
    new_note.offset=offset
    new_note.storedInstrument=instrument.Piano()
    output_notes.append(new_note)
    midi_stream=stream.Stream(output_notes)
    midi_stream.write('midi',fp='pred_music')


